# 趣味实战项目：AI 给文字区域打马赛克（视频）

解析文本区域检测器的结果，用 OpenCV 将文字区域打上马赛克。

指定输入视频路径，对视频中每一帧进行 OCR 和马赛克处理，输出打了马赛克的新视频。


## 进入MMOCR 主目录

In [1]:
import os
os.chdir('mmocr')
os.listdir()

['.git',
 '.circleci',
 '.codespellrc',
 '.coveragerc',
 '.github',
 '.gitignore',
 '.owners.yml',
 '.pre-commit-config.yaml',
 '.pylintrc',
 '.readthedocs.yml',
 'CITATION.cff',
 'LICENSE',
 'MANIFEST.in',
 'README.md',
 'README_zh-CN.md',
 'configs',
 'demo',
 'docker',
 'docs',
 'mmocr',
 'model-index.yml',
 'requirements.txt',
 'requirements',
 'resources',
 'setup.cfg',
 'setup.py',
 'tests',
 'tools',
 'mmocr.egg-info',
 'data',
 'font.TTF',
 'INFO',
 'outputs']

# 导入工具包

In [2]:
import numpy as np

# 导入 opencv-python
import cv2

from mmocr.utils.ocr import MMOCR

detector = MMOCR(det='TextSnake',  # 文本区域检测算法 
                 recog=None,       # 文本内容识别算法，这里不指定
                 device='cuda')     # 指定运算设备为 cpu 或 cuda

/environment/miniconda3/lib/python3.7/site-packages/mmdet/datasets/pipelines/formating.py:7: UserWarning: DeprecationWarning: mmdet.datasets.pipelines.formating will be deprecated, please replace it with mmdet.datasets.pipelines.formatting.
  warnings.warn('DeprecationWarning: mmdet.datasets.pipelines.formating will be '


load checkpoint from http path: https://download.openmmlab.com/mmocr/textdet/textsnake/textsnake_r50_fpn_unet_1200e_ctw1500-27f65b64.pth


# 处理单帧的函数

In [4]:
def process_frame(img):
    result = detector.readtext(img, output=None)

    for each in result[0]['boundary_result']: # 遍历每个文字区域
        if each[-1] > 0.2: # 置信度阈值
            point_coords = np.array(each[:-1]).reshape(-1,2)
            point_coords = np.array([point_coords], dtype = np.int32)
            cv2.fillPoly(img, point_coords, (0,255,0)) # 多边形填充
        else:
            print('有一片文字区域置信度为{:.2f}，小于阈值，不打码'.format(each[-1]))
    return img

# 视频逐帧处理（代码模板）¶

In [5]:
# 视频逐帧处理代码模板
# 不需修改任何代码，只需定义 process_frame 函数即可
from tqdm import tqdm

def generate_video(input_path='data/three-hands.mp4', output_path='data/output.mp4'):
    
    print('视频开始处理',input_path)
    
    # 获取视频总帧数
    cap = cv2.VideoCapture(input_path)
    frame_count = 0
    while(cap.isOpened()):
        success, frame = cap.read()
        frame_count += 1
        if not success:
            break
    cap.release()
    print('视频总帧数为',frame_count)
    
    cap = cv2.VideoCapture(input_path)
    frame_size = (cap.get(cv2.CAP_PROP_FRAME_WIDTH), cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS)

    out = cv2.VideoWriter(output_path, fourcc, fps, (int(frame_size[0]), int(frame_size[1])))
    
    # 进度条绑定视频总帧数
    with tqdm(total=frame_count-1) as pbar:
        try:
            while(cap.isOpened()):
                success, frame = cap.read()
                if not success:
                    break

                # 处理帧
                try:
                    frame = process_frame(frame)
                except Exception as e:
                    print(e)
                    pass
                
                if success == True:
                    out.write(frame)

                    # 进度条更新一帧
                    pbar.update(1)

        except:
            print('中途中断')
            pass

    cv2.destroyAllWindows()
    out.release()
    cap.release()
    print('视频已保存', output_path)

# 下载用于测试的视频素材

In [6]:
!wget https://download.openmmlab.com/mmocr/tutorial/test.mp4 -O data/test.mp4

--2022-03-28 12:59:21--  https://download.openmmlab.com/mmocr/tutorial/test.mp4
Connecting to 172.16.0.13:5848... connected.
Proxy request sent, awaiting response... 200 OK
Length: 7222265 (6.9M) [video/mp4]
Saving to: ‘data/test.mp4’

data/test.mp4       100%[===================>]   6.89M  --.-KB/s    in 0.07s   

2022-03-28 12:59:21 (99.3 MB/s) - ‘data/test.mp4’ saved [7222265/7222265]



# 在视频上执行预测

In [7]:
generate_video(input_path='data/test.mp4', output_path='outputs/output.mp4')

视频开始处理 data/test.mp4
视频总帧数为 78


  0%|          | 0/77 [00:00<?, ?it/s]/environment/miniconda3/lib/python3.7/site-packages/mmdet/datasets/utils.py:70: UserWarning: "ImageToTensor" pipeline is replaced by "DefaultFormatBundle" for batch inference. It is recommended to manually replace it in the test data pipeline in your config file.
  'data pipeline in your config file.', UserWarning)
100%|██████████| 77/77 [01:42<00:00,  1.33s/it]

视频已保存 outputs/output.mp4
